# Dimensión fecha

### Librerías necesarias

In [123]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

### Conexión con la base de datos

In [124]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [137]:
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn, columns=['id', 'fecha', 'hora'])
dim_fecha = table_estadosservicio
dim_fecha.set_index(dim_fecha.columns[0], inplace=True)
# dim_fecha = dim_fecha
dim_fecha.head()

,fecha,hora
id,,
1014,2024-01-29,01:13:32
1484,2024-01-30,18:45:12
2829,2024-02-06,11:34:04
1888,2024-02-01,14:50:39
32312,2024-04-06,16:11:21


### Módulo de transformación

In [138]:
dim_fecha["día"] = dim_fecha["fecha"].dt.day
dim_fecha["mes"] = dim_fecha["fecha"].dt.month
dim_fecha["año"] = dim_fecha["fecha"].dt.year
dim_fecha["día_semana"] = dim_fecha["fecha"].dt.weekday
dim_fecha["trimestre"] = dim_fecha["fecha"].dt.quarter

dim_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre
id,,,,,,,
1014,2024-01-29,01:13:32,29,1,2024,0,1
1484,2024-01-30,18:45:12,30,1,2024,1,1
2829,2024-02-06,11:34:04,6,2,2024,1,1
1888,2024-02-01,14:50:39,1,2,2024,3,1
32312,2024-04-06,16:11:21,6,4,2024,5,2


In [139]:
dim_fecha["día_del_año"] = dim_fecha["fecha"].dt.day_of_year
#dim_fecha["día_del_mes"] = dim_fecha["fecha"].dt.days_in_month
#dim_fecha["nombre_mes"] = dim_fecha["fecha"].dt.month_name()
dim_fecha["nombre_mes"] = dim_fecha["fecha"].dt.strftime('%B')
dim_fecha["nombre_día"] = dim_fecha["fecha"].dt.day_name()
dim_fecha['nombre_día'] = dim_fecha["nombre_día"].replace({'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles', 'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'})
#dim_fecha["str_fecha"] = dim_fecha["fecha"].dt.strftime("%d/%m/%Y")

dim_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día
id,,,,,,,,,,
1014,2024-01-29,01:13:32,29,1,2024,0,1,29,enero,lunes
1484,2024-01-30,18:45:12,30,1,2024,1,1,30,enero,martes
2829,2024-02-06,11:34:04,6,2,2024,1,1,37,febrero,martes
1888,2024-02-01,14:50:39,1,2,2024,3,1,32,febrero,jueves
32312,2024-04-06,16:11:21,6,4,2024,5,2,97,abril,sábado


In [140]:
co_holidays = holidays.CO(language="es")
dim_fecha["es_festivo"] = dim_fecha["fecha"].apply(lambda x:  x in co_holidays)
dim_fecha["festivo"] = dim_fecha["fecha"].apply(lambda x: co_holidays.get(x))
dim_fecha["fin_de_semana"] = dim_fecha["día_semana"].apply(lambda x: x>4)
dim_fecha['fecha'] = dim_fecha['fecha'].dt.date
dim_fecha.head()

,fecha,hora,día,mes,año,día_semana,trimestre,día_del_año,nombre_mes,nombre_día,es_festivo,festivo,fin_de_semana
id,,,,,,,,,,,,,
1014,2024-01-29,01:13:32,29,1,2024,0,1,29,enero,lunes,False,None,False
1484,2024-01-30,18:45:12,30,1,2024,1,1,30,enero,martes,False,None,False
2829,2024-02-06,11:34:04,6,2,2024,1,1,37,febrero,martes,False,None,False
1888,2024-02-01,14:50:39,1,2,2024,3,1,32,febrero,jueves,False,None,False
32312,2024-04-06,16:11:21,6,4,2024,5,2,97,abril,sábado,False,None,True


### Módulo de carga a la bodega de datos

In [141]:
try: # Realizar una consulta sencilla para verificar la conexión
    # Ejecutar una consulta para obtener el nombre de las tablas
    tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", olap_conn)
    #print(tables)
    print("Conexión establecida y consulta realizada con éxito.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión establecida y consulta realizada con éxito.


In [142]:
try:
  dim_fecha.to_sql('dim_fecha', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")